In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # You can use 'yolov8s.pt' or 'yolov8m.pt' for better accuracy

# Define base paths
base_path = r"C:\Users\ANNAPURNA\ANNU\ML PROJECTS\VisDrone2019-VID-train"
sequences_path = os.path.join(base_path, "sequences")
annotations_path = os.path.join(base_path, "annotations")

# Get all available sequence folders
sequence_folders = sorted(os.listdir(sequences_path))
print("Available sequence folders:", sequence_folders)

if not sequence_folders:
    raise FileNotFoundError("No sequence folders found in the given path.")

# Pick the first sequence folder
first_sequence_folder = sequence_folders[20]
first_sequence = os.path.join(sequences_path, first_sequence_folder)
first_annotation = os.path.join(annotations_path, first_sequence_folder + ".txt")

# Function to read annotation file
def read_annotation(annotation_file):
    annotations = {}
    if not os.path.exists(annotation_file):
        print(f"Annotation file '{annotation_file}' not found.")
        return annotations

    with open(annotation_file, "r") as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) < 8:
                continue  # Skip malformed lines
            frame_id = int(values[0])
            obj_id = int(values[1])
            x, y, w, h = map(int, values[2:6])
            class_id = int(values[7])
            if frame_id not in annotations:
                annotations[frame_id] = []
            annotations[frame_id].append((obj_id, x, y, w, h, class_id))
    return annotations

# Function to process video sequence
def process_sequence(sequence_folder, annotation_file):
    images = sorted(os.listdir(sequence_folder))
    annotations = read_annotation(annotation_file)

    # Prepare for video saving
    first_img_path = os.path.join(sequence_folder, images[0])
    first_img = cv2.imread(first_img_path)
    height, width = first_img.shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("output_detection.mp4", fourcc, 20.0, (width, height))

    for img_name in images:
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        frame_id = int(os.path.splitext(img_name)[0])
        img_path = os.path.join(sequence_folder, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Warning: Could not load image {img_path}")
            continue

        # Run YOLO detection
        results = model(img)

        # Iterate over each result (detected object)
        for result in results:
            for box, cls in zip(result.boxes.xyxy, result.boxes.cls):
                x1, y1, x2, y2 = map(int, box[:4])

                # Assign labels: "person" if class_id == 0, otherwise "vehicle"
                label = "person" if int(cls) == 0 else "vehicle"
                
                # Draw rectangle and label
                color = (0, 255, 0) if label == "person" else (255, 0, 0)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Draw ground truth annotations (blue)
        if frame_id in annotations:
            for obj in annotations[frame_id]:
                obj_id, x, y, w, h, class_id = obj
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(img, f"GT-ID:{obj_id}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # Show frame
        cv2.imshow("Detection", img)
        out.write(img)

        # Exit on 'q'
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

    out.release()
    cv2.destroyAllWindows()
    print("Video saved as output_detection.mp4")

# Run on the first sequence
process_sequence(first_sequence, first_annotation)


Available sequence folders: ['uav0000013_00000_v', 'uav0000013_01073_v', 'uav0000013_01392_v', 'uav0000020_00406_v', 'uav0000071_03240_v', 'uav0000072_04488_v', 'uav0000072_05448_v', 'uav0000072_06432_v', 'uav0000076_00720_v', 'uav0000079_00480_v', 'uav0000084_00000_v', 'uav0000099_02109_v', 'uav0000124_00944_v', 'uav0000126_00001_v', 'uav0000138_00000_v', 'uav0000140_01590_v', 'uav0000143_02250_v', 'uav0000145_00000_v', 'uav0000150_02310_v', 'uav0000218_00001_v', 'uav0000222_03150_v', 'uav0000239_03720_v', 'uav0000239_12336_v', 'uav0000243_00001_v', 'uav0000244_01440_v', 'uav0000248_00001_v', 'uav0000263_03289_v', 'uav0000264_02760_v', 'uav0000266_03598_v', 'uav0000266_04830_v', 'uav0000270_00001_v', 'uav0000273_00001_v', 'uav0000278_00001_v', 'uav0000279_00001_v', 'uav0000281_00460_v', 'uav0000288_00001_v', 'uav0000289_00001_v', 'uav0000289_06922_v', 'uav0000295_02300_v', 'uav0000300_00000_v', 'uav0000307_00000_v', 'uav0000308_00000_v', 'uav0000308_01380_v', 'uav0000309_00000_v', 'ua